In [ ]:
import pandas as pd
import requests

from fantasy import fantasy_points
from creds import nfl_api_key

In [ ]:
schedule_url = 'https://profootballapi.com/schedule'
game_url = 'https://profootballapi.com/game'

In [200]:
all_games = requests.post(schedule_url, params={'api_key': nfl_api_key, 'season_type': 'REG'}).json()

## Gather defensive data

In [173]:
curr_year = 2019
# defensive_data = []
try:
    for game in all_games:
        new_response = requests.post(game_url, params={'api_key': nfl_api_key, 'game_id': game['id']})
        game_resp = new_response.json()
        if str(curr_year) == game['year']:
            print(curr_year)
            curr_year += 1

        for team, loc, opposition_loc in [(game['home'], 'home', 'away'), (game['away'], 'away', 'home')]:
            opp_pyds = game_resp[opposition_loc]['pyds']
            opp_ryds = game_resp[opposition_loc]['ryds']
            opp_pts = game[opposition_loc + '_score']

            # Calculate touchdowns
            rtds = 0
            ptds = 0
            passing_players = game_resp[opposition_loc]['stats']['passing']
            for passing_player in passing_players.keys():
                ptds += passing_players[passing_player]['touchdowns']

            rushing_players = game_resp[opposition_loc]['stats']['rushing']
            for rushing_player in rushing_players.keys():
                rtds += rushing_players[rushing_player]['touchdowns']

            # Calculate specific defense stats
            forced_fumbles = 0
            ints = 0
            sacks = 0
            defensive_players = game_resp[loc]['stats']['defense']
            for defensive_player in defensive_players.keys():
                forced_fumbles += defensive_players[defensive_player]['forced_fumbles']
                ints += defensive_players[defensive_player]['interceptions']
                sacks += defensive_players[defensive_player]['sacks']
            defensive_data.append([game['year'], game['week'], team, opp_pyds, opp_ryds, opp_pts, forced_fumbles, ints, sacks])
except Exception as e:
    print('Error! {} vs {}. Year {} Week {}'.format(game['home'], game['away'], game['year'], game['week']))
    pass

In [183]:
defense_df = pd.DataFrame(defensive_data, columns=['Year', 'Week', 'Team', 'Opp Pass Yards', 'Opp Rush Yards', 'Opp Points', 'Forced Fumbles', 'Ints', 'Sacks'])
defense_df = defense_df.drop_duplicates()
defense_df['Year'] = defense_df['Year'].astype(int)
defense_df['Week'] = defense_df['Week'].astype(int)

In [193]:
defense_df.to_csv('defensive_data.csv', index=False)

### Individual Defense Stats

In [240]:
ind_defense_encountered = []

In [241]:
curr_year = 2009
ind_def_data = []

for game in all_games:
    try:
        game_id = str(game['year']) + str(game['week']) + game['home'] + game['away']
        if game_id in ind_defense_encountered:
            continue
        
        ind_defense_encountered.append(game_id)
            
        new_response = requests.post(game_url, params={'api_key': nfl_api_key, 'game_id': game['id']})
        game_resp = new_response.json()
        if str(curr_year) == game['year']:
            print(curr_year)
            curr_year += 1

        for team, loc, opp_loc in [(game['home'], 'home', 'away'), (game['away'], 'away', 'home')]:
            team_pts = game[loc + '_score']
            opp_team = game[opp_loc]
            opp_pts = game[opp_loc + '_score']

            # Gather player data
            players = {}

            if 'defense' in game_resp[loc]['stats'].keys():
                defense_players = game_resp[loc]['stats']['defense']
                for defense_player in defense_players.keys():
                    curr_player = defense_players[defense_player]
                    player_entry = { 'two_points': 0 }
                    if defense_player in players.keys():
                        player_entry = players[defense_player]
                    player_entry['name'] = curr_player['name']
                    player_entry['tackles'] = curr_player['tackles']
                    player_entry['assisted_tackles'] = curr_player['assisted_tackles']
                    player_entry['sacks'] = curr_player['sacks']
                    player_entry['interceptions'] = curr_player['interceptions']
                    player_entry['forced_fumbles'] = curr_player['forced_fumbles']
                    players[defense_player] = player_entry

            for player_key in players.keys():
                player = players[player_key]
                ind_def_data.append([game['year'], game['week'], team, opp_team, player.get('name'),
                                       player.get('tackles', 0), player.get('assisted_tackles', 0), player.get('sacks', 0), player.get('interceptions', 0), player.get('forced_fumbles', 0),
                                       team_pts, opp_pts, player_key
                                       ])
    except Exception as e:
        print('Error! {} vs {}. Year {} Week {}'.format(game['home'], game['away'], game['year'], game['week']))
        print(getattr(e, 'message', repr(e)))
        pass

2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
Error! TEN vs WAS. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! CAR vs ATL. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! CLE vs CIN. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! DAL vs TB. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! DET vs MIN. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! NE vs BUF. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! NYJ vs GB. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! PHI vs HOU. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! LAC vs BAL. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! ARI vs LA. Year 2018 Week 16
JSONDecodeError('Expecting value: l

In [242]:
ind_def_df = pd.DataFrame(ind_def_data, columns=['Year', 'Week', 'Team', 'Opposing Team', 'Name', 'Tackles', 'Assisted Tackles', 'Sacks', 'Ints', 'Forced Fumbles', 'Team Score', 'Opposing Score', 'PlayerID'])
ind_def_df.to_csv('ind_defensive_data.csv', index=False)

## Offensive Stats

In [236]:
offensive_encountered = []

In [237]:
curr_year = 2009
offensive_data = []

for game in all_games:
    try:
        game_id = str(game['year']) + str(game['week']) + game['home'] + game['away']
        if game_id in offensive_encountered:
            continue
        
        offensive_encountered.append(game_id)
            
        new_response = requests.post(game_url, params={'api_key': nfl_api_key, 'game_id': game['id']})
        game_resp = new_response.json()
        if str(curr_year) == game['year']:
            print(curr_year)
            curr_year += 1

        for team, loc, opp_loc in [(game['home'], 'home', 'away'), (game['away'], 'away', 'home')]:
            team_pts = game[loc + '_score']
            opp_team = game[opp_loc]
            opp_pts = game[opp_loc + '_score']

            # Gather player data
            players = {}

            if 'passing' in game_resp[loc]['stats'].keys():
                passing_players = game_resp[loc]['stats']['passing']
                for passing_player in passing_players.keys():
                    curr_player = passing_players[passing_player]
                    player_entry = { 'two_points': 0 }
                    if passing_player in players.keys():
                        player_entry = players[passing_player]
                    player_entry['name'] = curr_player['name']
                    player_entry['pass_attempts'] = curr_player['attempts']
                    player_entry['pass_tds'] = curr_player['touchdowns']
                    player_entry['pass_yds'] = curr_player['yards']
                    player_entry['interceptions'] = curr_player['interceptions']
                    player_entry['two_points'] += curr_player['two_point_makes']
                    players[passing_player] = player_entry

            if 'rushing' in game_resp[loc]['stats'].keys():
                rushing_players = game_resp[loc]['stats']['rushing']
                for rushing_player in rushing_players.keys():
                    curr_player = rushing_players[rushing_player]
                    player_entry = { 'two_points': 0 }
                    if rushing_player in players.keys():
                        player_entry = players[rushing_player]
                    player_entry['name'] = curr_player['name']
                    player_entry['rush_attempts'] = curr_player['attempts']
                    player_entry['rush_tds'] = curr_player['touchdowns']
                    player_entry['rush_yds'] = curr_player['yards']
                    player_entry['rush_long'] = curr_player['long']
                    player_entry['two_points'] += curr_player['two_point_makes']
                    players[rushing_player] = player_entry

            if 'receiving' in game_resp[loc]['stats'].keys():
                receiving_players = game_resp[loc]['stats']['receiving']
                for receiving_player in receiving_players.keys():
                    curr_player = receiving_players[receiving_player]
                    player_entry = { 'two_points': 0 }
                    if receiving_player in players.keys():
                        player_entry = players[receiving_player]
                    player_entry['name'] = curr_player['name']
                    player_entry['receptions'] = curr_player['receptions']
                    player_entry['rec_tds'] = curr_player['touchdowns']
                    player_entry['rec_yds'] = curr_player['yards']
                    player_entry['rec_long'] = curr_player['long']
                    player_entry['two_points'] += curr_player['two_point_makes']
                    players[receiving_player] = player_entry

            if 'fumbles' in game_resp[loc]['stats'].keys():
                fumble_players = game_resp[loc]['stats']['fumbles']
                for fumble_player in fumble_players.keys():
                    curr_player = fumble_players[fumble_player]
                    player_entry = { 'two_points': 0 }
                    if fumble_player in players.keys():
                        player_entry = players[fumble_player]
                    player_entry['name'] = curr_player['name']
                    player_entry['fumbles'] = curr_player['total_fumbles']
                    player_entry['fumbles_lost'] = curr_player['fumbles_lost']
                    players[fumble_player] = player_entry

            if 'kick_return' in game_resp[loc]['stats'].keys():
                kick_return_players = game_resp[loc]['stats']['kick_return']
                for kick_return_player in kick_return_players.keys():
                    curr_player = kick_return_players[kick_return_player]
                    player_entry = { 'two_points': 0 }
                    if kick_return_player in players.keys():
                        player_entry = players[kick_return_player]
                    player_entry['name'] = curr_player['name']
                    player_entry['kr_tds'] = curr_player['touchdowns']
                    players[kick_return_player] = player_entry

            if 'punt_return' in game_resp[loc]['stats'].keys():
                punt_return_players = game_resp[loc]['stats']['punt_return']
                for punt_return_player in punt_return_players.keys():
                    curr_player = punt_return_players[punt_return_player]
                    player_entry = { 'two_points': 0 }
                    if punt_return_player in players.keys():
                        player_entry = players[punt_return_player]
                    player_entry['name'] = curr_player['name']
                    player_entry['pr_tds'] = curr_player['touchdowns']
                    players[punt_return_player] = player_entry

            for player_key in players.keys():
                player = players[player_key]
                offensive_data.append([game['year'], game['week'], team, opp_team, player.get('name'), player.get('pass_attempts', 0), player.get('pass_yds', 0), player.get('pass_tds', 0),
                                       player.get('interceptions', 0), player.get('rush_attempts', 0), player.get('rush_yds', 0), player.get('rush_tds', 0), player.get('rush_long', 0),
                                       player.get('receptions', 0), player.get('rec_yds', 0), player.get('rec_tds', 0), player.get('rec_long', 0), player.get('two_points', 0), player.get('fumbles', 0), 
                                       player.get('fumbles_lost', 0), player.get('kr_tds', 0), player.get('pr_tds', 0), team_pts, opp_pts, player_key
                                       ])
    except Exception as e:
        print('Error! {} vs {}. Year {} Week {}'.format(game['home'], game['away'], game['year'], game['week']))
        print(getattr(e, 'message', repr(e)))
        pass

2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
Error! TEN vs WAS. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! CAR vs ATL. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! CLE vs CIN. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! DAL vs TB. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! DET vs MIN. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! NE vs BUF. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! NYJ vs GB. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! PHI vs HOU. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! LAC vs BAL. Year 2018 Week 16
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Error! ARI vs LA. Year 2018 Week 16
JSONDecodeError('Expecting value: l

In [239]:
offensive_df = pd.DataFrame(offensive_data, columns=['Year', 'Week', 'Team', 'Opposing Team', 'Name', 'Pass Attempts', 'Pass Yards', 'Pass TDs', 'Ints', 'Rush Attempts', 'Rush Yards', 'Rush TDs', 'Rush Long', 'Receptions', 'Rec Yards', 'Rec TDs', 'Rec Long', 'Two Points', 'Fumbles', 'Fumbles Lost', 'KR TDs', 'PR TDs', 'Team Score', 'Opposing Score', 'PlayerID'])
offensive_df.to_csv('offensive_data.csv', index=False)

In [ ]:
game_resp['home']['stats']['defense']

## Schedule Data

In [255]:
game_encountered = []

In [256]:
game_data = []
for game in all_games:
    game_id = str(game['year']) + str(game['week']) + game['home'] + game['away']
    if game_id in game_encountered:
        continue
        
    game_encountered.append(game_id)
    
    game_data.append([game['year'], game['week'], game['home'], game['home_score'], 
                      game['away'], game['away_score'], game['month'], game['day'], game['time']])

In [257]:
game_df = pd.DataFrame(game_data, columns=['Year', 'Week', 'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Month', 'Day', 'Time'])
game_df.to_csv('schedule_data.csv', index=False)

## Weather Data

In [258]:
weather_url = 'http://history.openweathermap.org/data/2.5/history/city'
city_id = 3882428
type_call = 'hour'
start = 1252585800
cnt = 2
weather_api_key = '26aa50f82507f34fca4e6d212f2bd716'

In [261]:
requests.get(weather_url, params={ 'id': city_id, 'type': type_call, 'appid': weather_api_key,
                                  'start': start, 'cnt': cnt})

<Response [401]>